In [1]:
import pandas as pd
import numpy as np
import geopandas
import geopy
import geocoder
from geopy.geocoders import Nominatim
from datetime import datetime as dt

## Reading CSV's of the daily obtained csv from hourly data.( Datasets available at drive link)

In [2]:
df = pd.read_csv("DATA (22 SEP - 21 OCT).csv",index_col = 0)
print(df.shape)

(64718, 26)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64718 entries, 0 to 64717
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Station    64718 non-null  object 
 1   Date_time  64718 non-null  object 
 2   City       64718 non-null  object 
 3   State      64718 non-null  object 
 4   PM10_min   53814 non-null  float64
 5   PM10_max   53814 non-null  float64
 6   PM10_avg   53814 non-null  float64
 7   PM2.5_min  58287 non-null  float64
 8   PM2.5_max  58287 non-null  float64
 9   PM2.5_avg  58287 non-null  float64
 10  NO2_min    58688 non-null  float64
 11  NO2_max    58688 non-null  float64
 12  NO2_avg    58688 non-null  float64
 13  NH3_min    48498 non-null  float64
 14  NH3_max    48498 non-null  float64
 15  NH3_avg    48498 non-null  float64
 16  SO2_min    55542 non-null  float64
 17  SO2_max    55542 non-null  float64
 18  SO2_avg    55542 non-null  float64
 19  CO_min     60440 non-null  float64
 20  CO_max

In [4]:
l = df.loc[df['AM/PM'].isna()].index.to_list()
df['AM/PM'] = df['AM/PM'].fillna('PM')
l = df.loc[df['AM/PM'].isna()].index.to_list()
df['AM/PM']= df['AM/PM'].replace('PAM','PM')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64718 entries, 0 to 64717
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Station    64718 non-null  object 
 1   Date_time  64718 non-null  object 
 2   City       64718 non-null  object 
 3   State      64718 non-null  object 
 4   PM10_min   53814 non-null  float64
 5   PM10_max   53814 non-null  float64
 6   PM10_avg   53814 non-null  float64
 7   PM2.5_min  58287 non-null  float64
 8   PM2.5_max  58287 non-null  float64
 9   PM2.5_avg  58287 non-null  float64
 10  NO2_min    58688 non-null  float64
 11  NO2_max    58688 non-null  float64
 12  NO2_avg    58688 non-null  float64
 13  NH3_min    48498 non-null  float64
 14  NH3_max    48498 non-null  float64
 15  NH3_avg    48498 non-null  float64
 16  SO2_min    55542 non-null  float64
 17  SO2_max    55542 non-null  float64
 18  SO2_avg    55542 non-null  float64
 19  CO_min     60440 non-null  float64
 20  CO_max

## TimeStamps

In [6]:
df['TimeStamp'] = df['Date_time'] + ' ' +  df['AM/PM']
form = {}
for stamp in df.TimeStamp:
    t = dt.strptime(stamp,'%d-%m-%Y %I:%M:%S %p')
    t = t.strftime('%d-%m-%Y %H:%M:%S %p')
    #convert to timedelta
    t = dt.strptime(t,"%d-%m-%Y %H:%M:%S %p")
    form[stamp] = t

form
df['test'] = df['TimeStamp'].map(form)

In [7]:
# df.sort_values(by = 'test').tail(20)

In [8]:
df = df.drop(['AM/PM','Date_time','TimeStamp'],axis = 1)
df.rename(columns = {'test' : 'Time'},inplace = True)
print(df.shape)
df.head()

(64718, 25)


,Station,City,State,PM10_min,PM10_max,PM10_avg,PM2.5_min,PM2.5_max,PM2.5_avg,NO2_min,...,SO2_min,SO2_max,SO2_avg,CO_min,CO_max,CO_avg,OZONE_min,OZONE_max,OZONE_avg,Time
0,"Secretariat, Amaravati - APPCB",Amaravati,Andhra_Pradesh,11.0,41.0,22.0,6.0,27.0,14.0,18.0,...,10.0,30.0,23.0,11.0,36.0,22.0,16.0,41.0,27.0,2020-09-22 14:00:00
1,"Anand Kala Kshetram, Rajamahendravaram - APPCB",Rajamahendravaram,Andhra_Pradesh,14.0,36.0,20.0,3.0,18.0,10.0,5.0,...,5.0,7.0,6.0,2.0,100.0,20.0,14.0,53.0,37.0,2020-09-22 14:00:00
2,"Tirumala, Tirupati - APPCB",Tirupati,Andhra_Pradesh,12.0,38.0,27.0,10.0,30.0,20.0,6.0,...,4.0,9.0,7.0,27.0,42.0,33.0,17.0,30.0,26.0,2020-09-22 14:00:00
3,"GVM Corporation, Visakhapatnam - APPCB",Visakhapatnam,Andhra_Pradesh,36.0,84.0,61.0,23.0,58.0,39.0,36.0,...,1.0,20.0,10.0,13.0,25.0,19.0,9.0,36.0,21.0,2020-09-22 14:00:00
4,"Railway Colony, Guwahati - APCB",Guwahati,Assam,9.0,78.0,31.0,3.0,98.0,29.0,8.0,...,23.0,25.0,23.0,20.0,38.0,22.0,16.0,21.0,18.0,2020-09-22 14:00:00


In [9]:
cities = list(set(list(df.City)))
len(cities)

126

## Coordinates

In [10]:
locator = Nominatim(user_agent="mikkel",timeout = 100)
# coord = []
coord = {}
for i,city in enumerate(cities):
#     print(str(i+1)+'.'+city)
    lc = locator.geocode(city)
    try:
        
#         coord.append([lc.latitude,lc.longitude])
        coord[city] = [lc.latitude,lc.longitude]
        print(str(i+1)+'.'+ city + ' SUCCESS!')
    except AttributeError:
        print(str(i+1)+'.'+city + ' DATA not available')
#         coord.append([city +'not','available'])
    
coord

1.Kaithal SUCCESS!
2.Thiruvananthapuram SUCCESS!
3.Mumbai SUCCESS!
4.Panipat SUCCESS!
5.Kohima SUCCESS!
6.Pali SUCCESS!
7.Ludhiana SUCCESS!
8.Paravattani SUCCESS!
9.Ratlam SUCCESS!
10.Yamunanagar SUCCESS!
11.Greater Noida SUCCESS!
12.Amaravati SUCCESS!
13.Bulandshahr SUCCESS!
14.Howrah SUCCESS!
15.Vapi SUCCESS!
16.Solapur SUCCESS!
17.Kollam SUCCESS!
18.Bagalkot SUCCESS!
19.Bathinda SUCCESS!
20.Chikkaballapur SUCCESS!
21.Coimbatore SUCCESS!
22.Gurugram SUCCESS!
23.Gandhinagar SUCCESS!
24.Vatva SUCCESS!
25.Thane SUCCESS!
26.Baghpat SUCCESS!
27.Rupnagar SUCCESS!
28.Kota SUCCESS!
29.Aurangabad SUCCESS!
30.Mandikhera SUCCESS!
31.Varanasi SUCCESS!
32.Damoh SUCCESS!
33.Rajamahendravaram SUCCESS!
34.Bhiwani SUCCESS!
35.Ujjain SUCCESS!
36.Kannur SUCCESS!
37.Sirsa SUCCESS!
38.Bhiwadi SUCCESS!
39.Kurukshetra SUCCESS!
40.Jaipur SUCCESS!
41.Jabalpur SUCCESS!
42.Maihar SUCCESS!
43.Yadgir SUCCESS!
44.Muzaffarpur SUCCESS!
45.Moradabad SUCCESS!
46.Singrauli SUCCESS!
47.Eloor SUCCESS!
48.Mandideep SUCCE

{'Kaithal': [29.7444561, 76.42555011591303],
 'Thiruvananthapuram': [8.5241122, 76.9360573],
 'Mumbai': [18.9387711, 72.8353355],
 'Panipat': [29.3912753, 76.9771675],
 'Kohima': [25.75, 94.166667],
 'Pali': [25.6040908, 73.41560878556902],
 'Ludhiana': [30.9090157, 75.851601],
 'Paravattani': [10.5190245, 76.241875],
 'Ratlam': [23.501957750000003, 74.95284518680097],
 'Yamunanagar': [30.2112003, 77.28638972360895],
 'Greater Noida': [28.4670734, 77.5137649],
 'Amaravati': [16.5744201, 80.3556079],
 'Bulandshahr': [28.38886085, 77.97479772598118],
 'Howrah': [22.5736296, 88.3251045],
 'Vapi': [20.3735482, 72.9084376],
 'Solapur': [17.84990665, 75.27632027348457],
 'Kollam': [8.8879509, 76.5955013],
 'Bagalkot': [16.1853166, 75.6967919],
 'Bathinda': [30.17911535, 75.04710156312628],
 'Chikkaballapur': [13.4352549, 77.7306866],
 'Coimbatore': [11.0018115, 76.9628425],
 'Gurugram': [28.4646148, 77.0299194],
 'Gandhinagar': [23.2232877, 72.6492267],
 'Vatva': [22.9589978, 72.6297981],
 '

In [8]:
cities.remove('Brajrajnagar')
print(len(cities))
len(coord)

125


125

In [11]:
times = list(set(list(df.Time)))
# # df.loc[df.Date_time == times[0]]
# times = list(set(list(df.Date_time)))
first_ind ={}
for time in times:
    first_ind[time]= df.loc[df.Time == time].index[0]
first_ind

{Timestamp('2020-10-16 09:00:00'): 46646,
 Timestamp('2020-09-30 10:00:00'): 8373,
 Timestamp('2020-10-19 10:00:00'): 54822,
 Timestamp('2020-10-18 19:00:00'): 54185,
 Timestamp('2020-10-14 22:00:00'): 42135,
 Timestamp('2020-10-21 05:00:00'): 60814,
 Timestamp('2020-10-13 14:00:00'): 37195,
 Timestamp('2020-10-17 12:00:00'): 51616,
 Timestamp('2020-10-20 13:00:00'): 59728,
 Timestamp('2020-10-14 14:00:00'): 40404,
 Timestamp('2020-10-21 15:00:00'): 62977,
 Timestamp('2020-10-19 11:00:00'): 55246,
 Timestamp('2020-10-02 07:00:00'): 12694,
 Timestamp('2020-10-02 15:00:00'): 14422,
 Timestamp('2020-10-18 13:00:00'): 52907,
 Timestamp('2020-09-28 13:00:00'): 4753,
 Timestamp('2020-10-05 12:00:00'): 21010,
 Timestamp('2020-10-02 18:00:00'): 15064,
 Timestamp('2020-10-08 09:00:00'): 28486,
 Timestamp('2020-10-18 09:00:00'): 52048,
 Timestamp('2020-10-17 22:00:00'): 51401,
 Timestamp('2020-10-18 14:00:00'): 53121,
 Timestamp('2020-10-20 16:00:00'): 60379,
 Timestamp('2020-10-21 20:00:00'): 6

In [12]:
len(first_ind)

300

In [13]:
list_df = []
i = 0
for time,ind in first_ind.items():
#     print(time,ind)
    a = df.loc[df['Time'] == df.Time[ind]].copy()
    list_df.append(a)
    i+=1
# a.TimeStamp.value_counts()

In [14]:
sum = 0
for _ in list_df:
    sum += _.shape[0]
print(sum)

64718


In [15]:
list_df[0].head()

,Station,City,State,PM10_min,PM10_max,PM10_avg,PM2.5_min,PM2.5_max,PM2.5_avg,NO2_min,...,SO2_min,SO2_max,SO2_avg,CO_min,CO_max,CO_avg,OZONE_min,OZONE_max,OZONE_avg,Time
46646,"Secretariat, Amaravati - APPCB",Amaravati,Andhra_Pradesh,16.0,60.0,36.0,13.0,52.0,30.0,22.0,...,12.0,31.0,20.0,23.0,57.0,35.0,19.0,61.0,25.0,2020-10-16 09:00:00
46647,"Anand Kala Kshetram, Rajamahendravaram - APPCB",Rajamahendravaram,Andhra_Pradesh,21.0,87.0,43.0,18.0,70.0,36.0,9.0,...,4.0,12.0,8.0,12.0,102.0,28.0,3.0,60.0,11.0,2020-10-16 09:00:00
46648,"Tirumala, Tirupati - APPCB",Tirupati,Andhra_Pradesh,17.0,50.0,29.0,8.0,36.0,22.0,8.0,...,1.0,12.0,6.0,36.0,64.0,44.0,19.0,118.0,56.0,2020-10-16 09:00:00
46649,"GVM Corporation, Visakhapatnam - APPCB",Visakhapatnam,Andhra_Pradesh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-16 09:00:00
46650,"Railway Colony, Guwahati - APCB",Guwahati,Assam,33.0,174.0,75.0,NaN,NaN,NaN,7.0,...,23.0,26.0,25.0,22.0,82.0,37.0,16.0,29.0,20.0,2020-10-16 09:00:00


## Dropping out stations. 
## Only City names are used.

In [16]:
num_cols = [cname for cname in df.columns if df[cname].dtype in ['int64','float64']]
city_set = list(set(cities))
def Cleaner(df):    
        df.drop('Station',axis = 1,inplace = True)
        df.sort_values(by = 'City',inplace = True )        

In [17]:
for _ in list_df:
#     print(df_t.shape)
    Cleaner(_)

In [18]:
list_df[0].head()

,City,State,PM10_min,PM10_max,PM10_avg,PM2.5_min,PM2.5_max,PM2.5_avg,NO2_min,NO2_max,...,SO2_min,SO2_max,SO2_avg,CO_min,CO_max,CO_avg,OZONE_min,OZONE_max,OZONE_avg,Time
46825,Agra,Uttar_Pradesh,NaN,NaN,NaN,102.0,318.0,235.0,10.0,84.0,...,NaN,NaN,NaN,16.0,86.0,32.0,78.0,237.0,106.0,2020-10-16 09:00:00
46698,Ahmedabad,Gujarat,61.0,164.0,109.0,58.0,245.0,130.0,25.0,131.0,...,8.0,56.0,30.0,8.0,107.0,36.0,2.0,158.0,7.0,2020-10-16 09:00:00
46804,Ajmer,Rajasthan,110.0,140.0,123.0,96.0,286.0,184.0,12.0,82.0,...,1.0,10.0,5.0,38.0,106.0,43.0,22.0,125.0,58.0,2020-10-16 09:00:00
46805,Alwar,Rajasthan,63.0,156.0,100.0,72.0,171.0,94.0,18.0,117.0,...,12.0,26.0,15.0,18.0,108.0,38.0,9.0,95.0,22.0,2020-10-16 09:00:00
46646,Amaravati,Andhra_Pradesh,16.0,60.0,36.0,13.0,52.0,30.0,22.0,55.0,...,12.0,31.0,20.0,23.0,57.0,35.0,19.0,61.0,25.0,2020-10-16 09:00:00


In [19]:
def Final_Cleaning(df):
    row_f = []
    for city in city_set:
        row = {}
        row['city'] = city

        for col in num_cols:
            temp = df[df['City'] == city][col].dropna().value_counts().sum()
            row[col] = df.loc[df['City']== city][col].dropna().sum()/temp
            
        row_f.append(row)
    d = pd.DataFrame(row_f).sort_values(by = 'city')
    d['Coordinates'] = d.city.map(coord)
    d['Time'] = df.iloc[0].Time
    return d

In [20]:
for i in range(300):
    print("Currently doing" + str(i))
    list_df[i] = Final_Cleaning(list_df[i])

Currently doing0


D:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


Currently doing1
Currently doing2
Currently doing3
Currently doing4
Currently doing5
Currently doing6
Currently doing7
Currently doing8
Currently doing9
Currently doing10
Currently doing11
Currently doing12
Currently doing13
Currently doing14
Currently doing15
Currently doing16
Currently doing17
Currently doing18
Currently doing19
Currently doing20
Currently doing21
Currently doing22
Currently doing23
Currently doing24
Currently doing25
Currently doing26
Currently doing27
Currently doing28
Currently doing29
Currently doing30
Currently doing31
Currently doing32
Currently doing33
Currently doing34
Currently doing35
Currently doing36
Currently doing37
Currently doing38
Currently doing39
Currently doing40
Currently doing41
Currently doing42
Currently doing43
Currently doing44
Currently doing45
Currently doing46
Currently doing47
Currently doing48
Currently doing49
Currently doing50
Currently doing51
Currently doing52
Currently doing53
Currently doing54
Currently doing55
Currently doing56
C

In [21]:
final = pd.concat(list_df,ignore_index = True)

In [23]:
final.sort_values(by = 'Time')

,city,PM10_min,PM10_max,PM10_avg,PM2.5_min,PM2.5_max,PM2.5_avg,NO2_min,NO2_max,NO2_avg,...,SO2_max,SO2_avg,CO_min,CO_max,CO_avg,OZONE_min,OZONE_max,OZONE_avg,Coordinates,Time
20411,Yamunanagar,78.000000,170.000000,121.000000,59.000000,239.000000,134.000000,14.000000,75.000000,31.000000,...,7.000000,6.000000,19.000000,74.000000,22.000000,10.0,122.0,32.000000,"[30.2112003, 77.28638972360895]",2020-09-22 14:00:00
20377,Palwal,49.000000,168.000000,109.000000,15.000000,66.000000,37.000000,19.000000,23.000000,20.000000,...,5.000000,4.000000,6.000000,54.000000,10.000000,8.0,65.0,21.000000,"[28.12502575, 77.35831300773046]",2020-09-22 14:00:00
20376,Pali,46.000000,187.000000,103.000000,39.000000,190.000000,83.000000,13.000000,24.000000,20.000000,...,22.000000,9.000000,22.000000,56.000000,37.000000,1.0,50.0,22.000000,"[25.6040908, 73.41560878556902]",2020-09-22 14:00:00
20375,Noida,54.000000,188.500000,124.750000,31.333333,197.333333,106.666667,12.500000,38.000000,21.250000,...,51.000000,15.666667,50.500000,94.000000,57.000000,7.5,89.0,19.750000,"[28.5356329, 77.3910727]",2020-09-22 14:00:00
20374,Navi Mumbai,NaN,NaN,NaN,13.000000,61.000000,43.000000,5.000000,33.000000,18.000000,...,13.000000,11.000000,72.000000,106.500000,79.000000,1.0,11.5,2.000000,"[19.0308262, 73.0198537]",2020-09-22 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35694,Fatehabad,117.000000,231.000000,176.000000,109.000000,332.000000,272.000000,4.000000,13.000000,8.000000,...,46.000000,27.000000,9.000000,63.000000,36.000000,30.0,195.0,98.000000,"[27.02766825, 78.2074341134146]",2020-10-21 22:00:00
35693,Faridabad,119.333333,492.333333,318.000000,58.250000,436.500000,292.000000,42.666667,132.000000,83.333333,...,47.750000,24.750000,32.750000,127.000000,69.750000,6.0,121.5,36.750000,"[28.4029786, 77.3103837]",2020-10-21 22:00:00
35692,Ernakulam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,68.000000,77.000000,74.000000,NaN,NaN,NaN,"[9.9825665, 76.2990313]",2020-10-21 22:00:00
35705,Howrah,21.000000,50.000000,35.333333,13.333333,42.333333,23.000000,21.666667,42.666667,28.333333,...,21.333333,13.333333,4.333333,35.333333,11.333333,10.0,30.0,24.333333,"[22.5736296, 88.3251045]",2020-10-21 22:00:00


In [24]:
final.to_csv('FINAL_DATA.csv')

In [27]:
df = pd.read_csv('FINAL_DATA.csv',index_col=0).sort_values(by = 'Time')

In [30]:
# cities = list(set(df.))
df.columns

Index(['city', 'PM10_min', 'PM10_max', 'PM10_avg', 'PM2.5_min', 'PM2.5_max',
       'PM2.5_avg', 'NO2_min', 'NO2_max', 'NO2_avg', 'NH3_min', 'NH3_max',
       'NH3_avg', 'SO2_min', 'SO2_max', 'SO2_avg', 'CO_min', 'CO_max',
       'CO_avg', 'OZONE_min', 'OZONE_max', 'OZONE_avg', 'Coordinates', 'Time'],
      dtype='object')

In [31]:
cols = ['city','Coordinates','Time','PM10_min', 'PM10_max', 'PM10_avg', 'PM2.5_min', 'PM2.5_max',
       'PM2.5_avg', 'NO2_min', 'NO2_max', 'NO2_avg', 'NH3_min', 'NH3_max',
       'NH3_avg', 'SO2_min', 'SO2_max', 'SO2_avg', 'CO_min', 'CO_max',
       'CO_avg', 'OZONE_min', 'OZONE_max', 'OZONE_avg']
df = df[cols]

In [33]:
df.rename(columns = {'city' : 'City'},inplace = True)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37800 entries, 20411 to 35721
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         37800 non-null  object 
 1   Coordinates  37500 non-null  object 
 2   Time         37800 non-null  object 
 3   PM10_min     31697 non-null  float64
 4   PM10_max     31697 non-null  float64
 5   PM10_avg     31697 non-null  float64
 6   PM2.5_min    32938 non-null  float64
 7   PM2.5_max    32938 non-null  float64
 8   PM2.5_avg    32938 non-null  float64
 9   NO2_min      33179 non-null  float64
 10  NO2_max      33179 non-null  float64
 11  NO2_avg      33179 non-null  float64
 12  NH3_min      28381 non-null  float64
 13  NH3_max      28381 non-null  float64
 14  NH3_avg      28381 non-null  float64
 15  SO2_min      32221 non-null  float64
 16  SO2_max      32221 non-null  float64
 17  SO2_avg      32221 non-null  float64
 18  CO_min       33719 non-null  float64
 19  

In [40]:
cities = sorted(list(set(df.City.to_list())))

In [41]:
cities

['Agra',
 'Ahmedabad',
 'Aizawl',
 'Ajmer',
 'Alwar',
 'Amaravati',
 'Ambala',
 'Amritsar',
 'Ankleshwar',
 'Asansol',
 'Aurangabad',
 'Bagalkot',
 'Baghpat',
 'Bahadurgarh',
 'Ballabgarh',
 'Bathinda',
 'Bengaluru',
 'Bhiwadi',
 'Bhiwani',
 'Bhopal',
 'Brajrajnagar',
 'Bulandshahr',
 'Chandigarh',
 'Chandrapur',
 'Charkhi Dadri',
 'Chennai',
 'Chikkaballapur',
 'Chikkamagaluru',
 'Coimbatore',
 'Damoh',
 'Delhi',
 'Dewas',
 'Dharuhera',
 'Eloor',
 'Ernakulam',
 'Faridabad',
 'Fatehabad',
 'Gandhinagar',
 'Gaya',
 'Ghaziabad',
 'Greater Noida',
 'Gurugram',
 'Guwahati',
 'Gwalior',
 'Hajipur',
 'Hapur',
 'Hisar',
 'Howrah',
 'Hubballi',
 'Hyderabad',
 'Indore',
 'Jabalpur',
 'Jaipur',
 'Jalandhar',
 'Jind',
 'Jodhpur',
 'Kaithal',
 'Kalaburgi',
 'Kalyan',
 'Kannur',
 'Kanpur',
 'Karnal',
 'Katni',
 'Khanna',
 'Kochi',
 'Kohima',
 'Kolkata',
 'Kollam',
 'Kota',
 'Kozhikode',
 'Kurukshetra',
 'Lucknow',
 'Ludhiana',
 'Maihar',
 'Mandi Gobindgarh',
 'Mandideep',
 'Mandikhera',
 'Manesar',

## These cities donot have enough data

In [42]:
cities_to_drop = ['Kaithal','Pali', 'Aizawl','Brajrajnagar','Ernakulam','Jind','Yadgir']

In [52]:
inds = df.loc[df.City == cities_to_drop[6]].index.to_list()
df = df.drop(index= inds)
df = df.reset_index()
df = df.drop('index',axis = 1)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35700 entries, 0 to 35699
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         35700 non-null  object 
 1   Coordinates  35700 non-null  object 
 2   Time         35700 non-null  object 
 3   PM10_min     30252 non-null  float64
 4   PM10_max     30252 non-null  float64
 5   PM10_avg     30252 non-null  float64
 6   PM2.5_min    31558 non-null  float64
 7   PM2.5_max    31558 non-null  float64
 8   PM2.5_avg    31558 non-null  float64
 9   NO2_min      31780 non-null  float64
 10  NO2_max      31780 non-null  float64
 11  NO2_avg      31780 non-null  float64
 12  NH3_min      26905 non-null  float64
 13  NH3_max      26905 non-null  float64
 14  NH3_avg      26905 non-null  float64
 15  SO2_min      30820 non-null  float64
 16  SO2_max      30820 non-null  float64
 17  SO2_avg      30820 non-null  float64
 18  CO_min       32186 non-null  float64
 19  CO_m

In [67]:
df.isna().sum(axis = 1).sort_values(ascending = False)

15712    12
11790    12
2890     12
27564    12
12387    12
         ..
20354     0
20355     0
20356     0
20357     0
0         0
Length: 32711, dtype: int64

In [69]:
l = df.loc[df.isna().sum(axis = 1) == 12].index.to_list()
df = df.drop(index= l)
df = df.reset_index()
df = df.drop('index',axis = 1)

In [70]:
df.shape

(32431, 24)

In [75]:
df = df.sort_values(by = 'Time')

In [77]:
df = df.reset_index()
df = df.drop('index',axis = 1)

In [81]:
df.describe()

,PM10_min,PM10_max,PM10_avg,PM2.5_min,PM2.5_max,PM2.5_avg,NO2_min,NO2_max,NO2_avg,NH3_min,...,NH3_avg,SO2_min,SO2_max,SO2_avg,CO_min,CO_max,CO_avg,OZONE_min,OZONE_max,OZONE_avg
count,30063.000000,30063.000000,30063.000000,31459.000000,31459.000000,31459.000000,31534.000000,31534.000000,31534.000000,26840.000000,...,26840.000000,30616.000000,30616.000000,30616.000000,31216.000000,31216.000000,31216.000000,29882.000000,29882.000000,29882.000000
mean,69.576179,177.156812,112.140211,50.518897,180.535084,108.071291,14.909970,53.060224,29.079527,3.794735,...,5.307435,7.372881,29.217722,14.285302,20.868080,66.554747,37.670676,12.059028,72.379444,36.998097
std,43.998838,115.260319,66.230306,41.276472,119.176358,80.235538,10.561928,45.237949,21.636455,3.569360,...,4.837219,7.009569,28.467580,11.628594,18.489986,33.262472,23.045280,15.060170,50.716234,33.676420
min,1.000000,12.000000,7.000000,1.000000,5.000000,3.000000,1.000000,2.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,32.000000,95.000000,60.000000,21.464286,76.000000,44.500000,8.000000,22.500000,15.000000,1.000000,...,2.000000,3.000000,10.000000,6.000000,11.000000,40.000000,23.000000,3.000000,35.839286,14.000000
50%,63.000000,144.000000,102.000000,40.500000,153.000000,82.000000,13.000000,36.000000,23.000000,3.000000,...,4.000000,5.666667,19.500000,11.000000,17.000000,61.000000,33.000000,8.000000,63.000000,28.000000
75%,105.000000,226.000000,148.333333,68.000000,289.000000,151.366667,20.000000,73.000000,36.500000,5.000000,...,7.000000,10.000000,37.000000,19.000000,26.600000,93.000000,46.000000,15.000000,96.000000,49.333333
max,223.000000,500.000000,363.000000,344.000000,500.000000,387.000000,86.000000,422.000000,196.000000,57.000000,...,76.000000,78.000000,140.000000,92.000000,200.000000,200.000000,200.000000,185.000000,276.000000,276.000000


In [82]:
df.to_csv('final.csv')